# Performance em GPU

Veremos algumas estratégias para melhorar a perfomance em GPUs

### Precisão numérica

GPUs modernas realizam operações de forma muito mais eficiente em float16. Mas operações em float16 possuem menor precisão, então é preciso tomar cuidado com a estabilidade numérica dos resultados. Existem algumas abordagens para realizar operações em menor precisão e ao mesmo tempo evitar erros numéricos. 

Compararemos as seguintes situações que possuem diferentes relações custo x precisão:

1. Operações em float64, o que dá o resultado mais preciso possível, mas é menos eficiente
2. Operações em float32, que é o padrão do Pytorch
3. O Pytorch disponibiliza a chamada *automatic mixed precision*. Quando é detectado que uma operação pode ser realizada sem muita perda de precisão, o Pytorch automaticamente realiza a operação em menor precisão. Para isso, é usado o contexto `torch.autocast`
4. Realizar operações em float16, o que é extremamente eficiente em GPUs modernas. Mas é preciso tomar cuidado, por exemplo, não é recomendado realizar o backpropagation em float16
5. Em GPUs recentes o Pytorch possui a função `torch.set_float32_matmul_precision`, que permite o uso de um tipo especial de dado, o chamado *tensorfloat32*. Esse tipo de dado é usado exclusivamente por tensor cores. Ele permite uma precisão próxima de float32 mas com a eficiência de float16

Aplicaremos essas técnicas em multiplicações matriciais

In [3]:
import torch

class PerfRecorder:
    """Registra o tempo de execução na GPU e o uso de memória."""

    def __init__(self):

        self.gpu_start = torch.cuda.Event(enable_timing=True)
        self.gpu_end = torch.cuda.Event(enable_timing=True)  

    def start(self):
        """Inicia registro."""

        torch.cuda.reset_peak_memory_stats()
        self.gpu_start.record() 

    def end(self):
        """Encerra registro."""

        self.gpu_end.record()
        torch.cuda.synchronize()

        # Tempo de execução na GPU, em ms
        t_gpu = self.gpu_start.elapsed_time(self.gpu_end)
        # Uso de memória em GiB
        max_memory = torch.cuda.max_memory_allocated()/2**30
    
        return t_gpu, max_memory
    
def benchmark(shape, dtype, n=5, n_warm=2):
    '''Realiza `n` multiplicações matriciais entre matrizes de tamanho
    shape[0]xshape[1] x shape[1]xshape[0]. Retorna o tempo médio de cada 
    multiplicação, a memória utilizada e a média dos valores do resultado.'''

    recorder = PerfRecorder()

    nr, nc = shape

    torch.manual_seed(0)
    x1 = torch.randn(nr, nc, dtype=dtype, device='cuda')
    x2 = torch.randn(nc, nr, dtype=dtype, device='cuda')

    for _ in range(n_warm):
        _ = torch.matmul(x1, x2)

    recorder.start()
    for _ in range(n):
        r = torch.matmul(x1, x2)
    t_gpu, max_memory = recorder.end()
    t_gpu /= n

    # Média dos valores do resultado do cálculo
    mean_val = r.abs().mean().item()
    
    return t_gpu, max_memory, mean_val

# Memória disponível na GPU em GiB
mem_size = 12
# Quantidade de valores que podem ser alocados em float64
nv = (12-2)*2**30//8
# /2 porque vamos alocar duas matrizes
nv = nv//2
# Tamanho das matrizes. As dimensões terem ao menos tamanho 512 garante que
# a GPU será utilizada ao máximo
mat_shape = (512, nv//512)

In [2]:
# float64
t_f64, m_f64, v_f64 = benchmark(mat_shape, torch.float64)
# float32
t_f32, m_f32, v_f32 = benchmark(mat_shape, torch.float32)
# float16
t_f16, m_f16, v_f16 = benchmark(mat_shape, torch.float16)

print('Tempos (ms):')
print(f'float64: {t_f64:.1f}\nfloat32: {t_f32:.1f}\nfloat16: {t_f16:.1f}')

print('\nMemória (GiB):')
print(f'float64: {m_f64:.1f}\nfloat32: {m_f32:.1f}\nfloat16: {m_f16:.1f}')

print('\nMédia dos valores:')
print(f'float64: {v_f64:.3f}\nfloat32: {v_f32:.3f}\nfloat16: {v_f16:.3f}')

Tempos (ms):
float64: 1485.7
float32: 41.9
float16: 11.5

Memória (GiB):
float64: 10.0
float32: 5.0
float16: 2.5

Média dos valores:
float64: 915.227
float32: 913.180
float16: 913.000


Note as diferenças de tempo. Os cálculos demoram muito mais em float64, e a multiplicação em float16 é mais de 3x mais rápida do que em float32! Isso representa um potencial de speedup de mais de 3x somente modificando a precisão! Mas note que os resultados apresentam pequenas diferenças.

O Pytorch possui duas abordagens para realizar cálculos em meia precisão :

In [3]:
# Autocast para float16, em operações selecionadas. Multiplicação matricial
# é uma dessas operações
with torch.autocast(device_type='cuda', dtype=torch.float16):
    t_af16, m_af16, v_af16 = benchmark(mat_shape, torch.float32)

# Uso do formato tf32 (tensorfloat32) para realizar a multiplicação
torch.set_float32_matmul_precision('high')
t_tf32, m_tf32, v_tf32 = benchmark(mat_shape, torch.float32)

print('Tempos (ms):')
print(f'autocast: {t_af16:.1f}\ntensorfloat32: {t_tf32:.1f}')

print('\nMemória (GiB):')
print(f'autocast: {m_af16:.1f}\ntensorfloat32: {m_tf32:.1f}')

print('\nResultados:')
print(f'autocast: {v_af16:.3f}\ntensorfloat32: {v_tf32:.3f}')

Tempos (ms):
autocast: 23.8
tensorfloat32: 30.3

Memória (GiB):
autocast: 7.5
tensorfloat32: 5.0

Resultados:
autocast: 913.000
tensorfloat32: 912.999


O autocast e o formato tf32 permitem um speedup em relação à float32. 

Há também o formato bfloat32 que possui a mesma magnitude que float32 mas menos resolução. As informações sobre os formatos do Pytorch são:

In [5]:
def pformat(format):
    info = torch.finfo(format)
    print(f'{info.max=}, {info.smallest_normal=}, {info.eps=}')

pformat(torch.float32)
pformat(torch.float16)
pformat(torch.bfloat16)

info.max=3.4028234663852886e+38, info.smallest_normal=1.1754943508222875e-38, info.eps=1.1920928955078125e-07
info.max=65504.0, info.smallest_normal=6.103515625e-05, info.eps=0.0009765625
info.max=3.3895313892515355e+38, info.smallest_normal=1.1754943508222875e-38, info.eps=0.0078125


### Memória *page-locked*

É possível copiar dados da CPU para a GPU utilizando memória page-locked. Isso aumenta de forma significativa o desempenho da cópia.

In [4]:
def copy(mat_shape, pin):

    recorder = PerfRecorder()

    x = torch.rand(mat_shape)
    if pin:
        x = x.pin_memory()

    recorder.start()
    x = x.to('cuda')
    t_gpu, _ = recorder.end()
    
    return t_gpu

t_nopin = copy(mat_shape, False)
t_pin = copy(mat_shape, True)

print('Tempos (ms):')
print(f'pin=False: {t_nopin:.1f}\npin=True: {t_pin:.1f}')

Tempos (ms):
pin=False: 506.3
pin=True: 213.2


Copiar tensores com pin_memory habilitado é quase 2x mais rápido. Isso é muito relevante, considerando que copiar dados da CPU para a GPU é uma operação extremamente custosa (note que demora mais para copiar uma matriz do que para realizar a multiplicação matricial entre centenas de milhões de valores).

Os dataloaders do Pytorch possuem uma parâmetro `pin_memory` que quando True utiliza essa técnica.

### Checkpoint de gradiente

In [10]:
from torchvision import models

def bench_model(model, bs, n=5, n_warm=2):

    optim = torch.optim.SGD(model.parameters())
    x = torch.rand(bs, 3, 224, 224, device='cuda')
    recorder = PerfRecorder()

    for _ in range(n_warm):
        _ = model(x)

    recorder.start()
    for _ in range(n):
        scores = model(x)
        loss = scores.mean()
        loss.backward()
        optim.step()
    t_gpu, max_memory = recorder.end()

    return t_gpu, max_memory

resnet = models.resnet50().to('cuda')
vit = models.vit_b_16().to('cuda')

t_resnet, m_resnet = bench_model(resnet, bs=64)
t_vit, m_vit = bench_model(vit, bs=64)

print('Tempos (ms):')
print(f'ResNet: {t_resnet:.1f}\nViT: {t_vit:.1f}')

print('Memória (GiB):')
print(f'ResNet: {m_resnet:.1f}\nViT: {m_vit:.1f}')

Tempos (ms):
ResNet: 733.8
ViT: 2615.9
Memória (GiB):
ResNet: 5.9
ViT: 8.7


### Compilação do grafo de execução

Cada operação realizada pelo Pytorch na GPU envolve a chamada de uma função cuda. Isso representa um custo adicional de comunicação entre CPU e GPU. Adicionalmente, existem diversas operações que podem ser otimizadas. Por exemplo, uma operação de convolução seguida de uma operação batchnorm pode ser representada por uma única operação linear dado que ambas as operações são lineares. Um loop de treinamento envolve chamar exatamente as mesma funções diversas vezes. O Pytorch permite compilar um grafo de executação que otimizará uma sequência de operações realizadas na GPU. Isso é feito através da função `torch.compile()`. Por exemplo, dado um modelo, podemos fazer:

`model = torch.compile(model, fullgraph=True, dynamic=False)`

O modelo pode então ser utilizado no loop de treinamento. Isso pode levar a speedups significativos. Mas há restrições nos tipos de modelos que podem ser compilados.